### geostatistics generic workflow
proof of concept of a full geostatistics workflow using only python and free modules. Jupyter as Platform.
  
v1.0 2024/05 paulo.ernesto  
  
*Those auxiliary scripts and modules must be available:*
 - _gui.py
 - workflowform.py
 - pd_vtk.py
 - db_linear_model.py
 - shell_vulcan.py
 - wf01.yaml
 - panel module (`pip install panel`)
#### Notes
 - If the database is drillholes, it must be converted to samples. Generic tool: db_composite_runlength.bat
 - Its possible to add new fields to the input form by editing the .yaml file
 - The export BMF step will only generate a result if Maptek Vulcan is available

In [ ]:
import sys, os.path, param, random, yaml, re
import numpy as np
import pandas as pd
import holoviews as hv
import panel as pn
from IPython.display import Markdown
from workflowform import WorkFlowForm
from pd_vtk import pv_read, pv_save, vtk_mesh_info, vtk_samples_to_grid, vtk_mesh_to_df, vtk_array_ijk, vtk_linear_model_variables, vtk_krig_model_variables
from _gui import pd_detect_xyz
print('Python %d.%d.%d' % sys.version_info[:3])
hv.extension('plotly')
pn.extension('vtk')

In [ ]:
display(Markdown('### dynamic input form with file persistence'))
form = WorkFlowForm('wf01.yaml') 
pn.panel(form)

In [ ]:
display(Markdown(chr(10).join(['key|value','---|---'] + [f'{k}|{v}' for k,v in form.items()])))
df = pd.read_excel(form.get('sample_db'))
df.mask(df == -99, inplace=True)
xyz = pd_detect_xyz(df)
display(Markdown('### load database:  \n`%d records, %d fields`' % df.shape))

In [ ]:
display(Markdown('### sample checks'))
p = pn.layout.GridBox(ncols=2)
p.append(hv.Points(df, kdims=xyz[:2], label='xy samples').opts(color=xyz[2]))
for v in form.get('grade_fields'):
  p.append(hv.BoxWhisker(df, kdims=form.get('lito_field'), vdims=v, label=f'{v} database boxplot'))
p

In [ ]:
display(Markdown('### grid create'))
grid = vtk_samples_to_grid(df, str(form.get('grid_size')))
grid.cells_volume('volume')
print(vtk_mesh_info(grid))

In [ ]:
if len(form.get('lito_mesh')):
  display(Markdown('### flag lito solids'))
  from vtk_flag_regions import vtk_flag_region
  fn_litoname = lambda _: re.sub(r'.*_(.+)\.\w+', r'\1', _)
  vtk_flag_region(grid, list(map(pv_read, form.get('lito_mesh'))), form.get('lito_field'), True, list(map(fn_litoname , form.get('lito_mesh'))))
else:
  display(Markdown('### use default lito n'))
  df[form.get('lito_field')] = 'n'
  grid.cell_data[form.get('lito_field')] = np.full(grid.n_cells, 'n')

In [ ]:
display(Markdown('### grade lito check'))
p = pn.layout.GridBox(ncols=2)
p.append(pd.pivot_table(vtk_mesh_to_df(grid), 'volume', form.get('lito_field'), None, 'sum'))
for i in range(3):
  p.append(hv.Image(grid.heatmap2d(form.get('lito_field'), i), label=chr(88 + i)).opts(xaxis='bare', yaxis='bare'))
p

In [ ]:
display(Markdown('### estimate grades by lito'))
display(Markdown(f'`engine: {form.get("engine")}`'))
print(vtk_mesh_info(grid))
if form.get('engine') == 'pykrig':
  vtk_krig_model_variables(grid, df, form.get('grade_fields'), form.get('lito_field'), None)
if form.get('engine') == 'scikit':
  vtk_linear_model_variables(grid, df, form.get('grade_fields'), form.get('lito_field'))

In [ ]:
display(Markdown('### grid grade checks'))
display(pd.pivot_table(vtk_mesh_to_df(grid), form.get('grade_fields'), form.get('lito_field'), None, ['min','mean','max']))
p = pn.layout.GridBox(ncols=3)
for v in form.get('grade_fields'):
  for i in range(3):
    p.append(hv.Image(grid.heatmap2d(v, i), label=v + ' ' + chr(88 + i)).opts(xaxis='bare', yaxis='bare'))
p

In [ ]:
display(Markdown('### grid voxel view'))
for v in form.get('grade_fields'):
  display(Markdown(v))
  display(pn.pane.VTKVolume(vtk_array_ijk(grid, v)))

In [ ]:
display(Markdown('### grid reserves'))
df_reserves = pd.pivot_table(vtk_mesh_to_df(grid), ['volume'] + form.get('grade_fields'), form.get('lito_field'), None, ['sum','mean'])
xlsx = os.path.splitext(form.get('grid_file'))[0] + '.xlsx'
df_reserves.to_excel(xlsx)
df_reserves

In [ ]:
display(Markdown('### save grid to file: \n`%s`' % form.get('grid_file')))
grid.save(form.get('grid_file'))

### export to csv

In [ ]:
if form.get('export_csv'):
  pv_save(grid, os.path.splitext(form.get('grid_file'))[0] + '.csv')

### export to bmf (requires Maptek Vulcan)

In [ ]:
r = None
if form.get('export_bmf'):
  try:
    from shell_vulcan import eval_vulcan
  except:
    r = 'vulcan API not available'
  finally:
    r = eval_vulcan('from _gui import pyd_zip_extract; pyd_zip_extract("shell_vulcan.pyz"); from pd_vtk import vtk_Voxel; vtk_Voxel.factory("%s").to_bmf("%s")' % (form.get('grid_file'), os.path.splitext(form.get('grid_file'))[0] + '.bmf'))
else:
  r = 'export bmf disabled'
print(r)